

Everything that you've built up over the course of the **/week2_openai** is all about [OpenAI Agents SDK](https://openai.github.io/openai-agents-python/), formerly known as `SWARM`. But first, before we can do OpenAI Agents SDK, I have a sidebar, a very important topic, and it's to talk to you about asynchronous Python, AsyncIO. 

**Sidebar: let´s talk about `asyncio`**

- All the Agent Frameworks use `asynchronous python`
- You can get by ignoring it but it will always bother you slightly
- Bite the bullet"! Spend 30 minutes on the guide - you will thank me!

It's something which is common across all of the Agent frameworks, so all of them make use of asynchronous Python, and it's something which you can get by without really understanding. There's a couple of rules that you can learn, and then you just follow those rules always, and it will just kind of work, and that's okay, but I'm here to tell you that that's unsatisfactory, and that it's much better to just take half an hour to get to the bottom of this, to really understand it, to thrash it out until you're like, okay,  I get asynchronous Python, I understand it, and if you do this, if you take the half an hour, you will thank me. It will, again and again, you'll come across this, and you'll be completely comfortable with it, and it's only half an hour, and so I put a guide, and the guides that will take you through this, so that you can be left with no questions in your mind at all about what it means to write asynchronous Python, and I'm also now going to cover it just very briefly at a high level. Let's talk AsyncIO. 

**Short version**
<p align="center">
  <img src="img/01.png" width="600">
</p>


Well, look, first of all, there is like a short version, a simple version. This is what I mean when I say you can get by without really understanding it. So AsyncIO is a way of writing Python code, which is a kind of lightweight version of multithreading, so many people from a software engineering background will be familiar with the idea of multithreading. When you write code that can run concurrently, you can have multiple threads which are each executing code together, and there's often a lot of baggage that comes with that, some sort of framework stuff that comes with it. Well, AsyncIO, which was introduced, I think, in Python 3.5, is this very lightweight way of doing it, which doesn't actually involve threads at an operating system level, and it also doesn't involve what's called multiprocessor, which is when you spawn multiple Python processes, and they all run together. This is another way of doing it that is super lightweight, and because it's super lightweight, it means that you can have thousands or tens of thousands of these things all running without consuming much resources at all. 

So it's a simple, easy-peasy, lightweight way of writing code that can run concurrently, and particularly, it's good when you have code that makes use of input-output, like waiting on networks. It allows other things to be running whilst one bit of code is waiting on the network, and when you're running LLM requests, you're mostly, if you're using paid APIs like OpenAI, most of the time is being spent waiting for stuff to feed back from a model running on the cloud, so there's a lot of waiting on networks, a lot of IO-bound waiting, and as a result, asynchronous code is great to use, and when you're thinking of multi-agent frameworks, and potentially have lots of these all hitting different paid APIs, it makes so much sense to be using it, `3:46` 

and that is why all of the agent frameworks we'll look at use AsyncIO. Okay, so with that preamble, this is the short version. The short version is AsyncIO is actually two things. It's a package called AsyncIO that you can import, and it's also some language constructs baked into Python, and those language constructs include two keywords that you see right here, and one of those keywords is the word Async, and anytime that you have a function which is potentially going to be able to run in a way that allows other things to happen concurrently, you simply put the word Async before it, like this, Async def do some processing, so def do some processing becomes Async def do some processing. That's how you say this is a function which can run asynchronously, and when you call that function, you don't just call do some processing like you're used to, you have to put the word await before it, so you say await do some processing, and if do some processing returns a string, then you can't just say result equals do some processing. You have to say result equals await do some processing, but otherwise it's just the same, so that's the short version, and if you wanted to get by without really understanding it, you could just follow those rules and keep going, and it might be enough, but now let me tell you the bigger story. Okay, now we're going to go a little bit deeper and talk about the real story with AsyncIO. So look, as I say, it's a lightweight alternative to multi-threading or multi-processing. Something like multi-threading is implemented at the OS level, and it supports concurrent execution of different programs where the CPU at the CPU level has been managed to sort of switch between these two programs and treat them as if they're running at the same time. This is different. In the case of AsyncIO, so there are some special keywords. There's this keyword Async. If you define a function as AsyncDepth, and then the function, instead of just Depth, the function, then this thing is no longer actually called a function. It's known as a coroutine. Now, most people still use the word function. You'll hear me saying function, but strictly speaking, any time you see AsyncDepth, you should say, okay, that's not a function. It is a coroutine, and that means it's something special that Python can pause and resume. When you call a coroutine, it doesn't run unlike any other function that you call that does run. When you call a coroutine, it just returns a coroutine object, but nothing is executed. To actually run that coroutine object, there are a few ways to do it, but the most common one, and the one that we will use all the time, is to await it. So you say, await, and then the coroutine object, and that schedules it for execution. What does it mean to say it's scheduled for execution? Well, there is this piece of code written in the AsyncIO library, which does what's known as an event loop, which means it's a kind of while loop that sort of iterates, and it takes this coroutine, and it starts executing. Executing this coroutine, it takes this coroutine, and it starts executing it, and it can only execute one coroutine at a time. It's not like multi-threading. It's going to be executing this coroutine, except if this coroutine gets to a point when it's stopped, and it's waiting for something, like it's waiting for IO. Perhaps it's made a call to OpenAI, and it's waiting for OpenAI to respond. At that point, the event loop can put that on pause and start executing a different one of the coroutines that's in its sort of list of coroutines waiting to be run, and if that hits a point when it's waiting on IO, then the event loop can run another one, or it can continue the first one. So it's a sort of manual way of handling multi-threading. It's a manual approach to implementing multi-threading, but only really working when something is blocked waiting on IO, and because of this, because it's sort of written at a code level, and it's super simple and very easy to understand, it's also lightweight. You can have thousands or tens of thousands of these things, and it's really easy and quick to get this running, and that is why it's so popular, and that is why it's used so ubiquitously across agentic frameworks. So I hope, obviously, I've only given you a bit of detail here, but I hope that gives you some perspective on what's going on, and when we look back now at this, you can see that whilst a simple interpretation is just to say, okay, whenever I use async, I've got to use await. The deeper interpretation is to understand that when I say async depth do some processing, that's no longer a function, it's now a coroutine. Do some processing, we need to call await do some processing in order to schedule that coroutine, and that is now blocking until that completes, and the result will go into that variable result. I hope that made some sense, but if not, as I say, there's a whole guide, you should go and look at the guide now. And so just to beat this one to death, I'll give you a couple more examples, and then hopefully you've really got this. So look at this again, this is back to the async depth do some processing. It's going to do some work, and then it's going to return the string, done. If I just say like my coroutine equals do some processing, you might, you know, if you didn't know already about this, you might think that that will run do some processing, but of course it will not. It will return a coroutine object, and that's what will go into the variable my coroutine. In order to actually run it, you have to call await. So you'd say my result equals await my coroutine is now going to run, and the result, the string done, is going to go into my result. So that is the point, that's how it works, and of course you don't need to do, you can, it's simpler than that, you can just say...